In [84]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point

In [85]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [86]:
# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords})


In [87]:
# Assuming property_gdf and victoria_gdf are already defined as per your provided code...
victoria_gdf = gpd.read_file('../data/Statistical_area_level2/victoria.geojson')

# Perform a spatial join
property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")

/home/jqqian1/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_28908/2101387466.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")


In [89]:
property_SA2['name'] = property_SA2['name'].str[-4:]

In [90]:
property_SA2 = property_SA2[['name', 'geometry', 'SA2_CODE21', 'SA2_NAME21']]

In [91]:
property_SA2.rename(columns={'name': 'Postcode'}, inplace=True)

/tmp/ipykernel_28908/2595192371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_SA2.rename(columns={'name': 'Postcode'}, inplace=True)


In [92]:
property_SA2['Postcode'] = property_SA2['Postcode'].astype('int')

/home/jqqian1/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [93]:
offence = pd.read_excel("../data/external_SA2/Offence Count.xlsx", sheet_name='Table 03')

In [94]:
offence_count = offence.groupby('Postcode')['Offence Count'].sum().reset_index()

In [95]:
crime = pd.merge(property_SA2, offence_count, on='Postcode', how='left')

In [97]:
crime.to_csv("../data/raw/Offence count by postcode.csv")